# Environment Configuration

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
from itertools import product

from google.colab import auth
from google.cloud import bigquery
from gspread_dataframe import set_with_dataframe

import gspread
from oauth2client.client import GoogleCredentials
from google.auth import default
creds, _ = default()

import pandas_gbq
from tqdm import tqdm
import time
from datetime import datetime
from shapely.geometry import Point, Polygon
import time
import random
import json

auth.authenticate_user()
print('Authenticated')

# Project Configuration
project_id = 'database_name'
client = bigquery.Client(project=project_id)

# Google Map API Test Area



In [ ]:
import requests

f = open('/content/drive/MyDrive/google_key.json')
google_key = json.load(f)

# 設定Google Maps API金鑰
api_key = google_key['X-Goog-Api-Key_4']

# 設定起點和終點座標
origin = '37.419734,-122.0827784'
destination = '37.417670,-122.079595'

# 構建API請求URL
url = f'https://maps.googleapis.com/maps/api/directions/json?origin={origin}&destination={destination}&mode=driving&key={api_key}'

# 發送API請求
response = requests.get(url)
data = response.json()

# 提取距離（公里）和時間（分鐘）
if data['status'] == 'OK':
    route = data['routes'][0]
    leg = route['legs'][0]
    distance_in_km = leg['distance']['value'] / 1000  # 轉換成公里
    duration_in_minutes = leg['duration']['value'] / 60  # 轉換成分鐘
    print(f"Distance: {distance_in_km} km")
    print(f"Duration: {duration_in_minutes} minutes")
else:
    print(f"Error: {data['status']}")



In [ ]:
import requests
import json

# Load Google Maps API key
try:
    with open('/content/drive/MyDrive/google_key.json') as f:
        google_key = json.load(f)
    api_key = google_key['X-Goog-Api-Key_4']
except FileNotFoundError as e:
    print(f"Error: The file was not found. {e}")
    exit()
except KeyError as e:
    print(f"Error: The key was not found in the JSON file. {e}")
    exit()
except json.JSONDecodeError as e:
    print(f"Error: JSON decoding failed. {e}")
    exit()

# Set origin and destination coordinates
origin = '37.419734,-122.0827784'
destination = '37.417670,-122.079595'

# Build API request URL
url = f'https://maps.googleapis.com/maps/api/directions/json?origin={origin}&destination={destination}&mode=driving&key={api_key}'

try:
    # Send API request
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for HTTP errors
    data = response.json()

    # Extract distance (km) and duration (minutes)
    if data['status'] == 'OK':
        route = data['routes'][0]
        leg = route['legs'][0]
        distance_in_km = leg['distance']['value'] / 1000  # Convert to kilometers
        duration_in_minutes = leg['duration']['value'] / 60  # Convert to minutes
        print(f"Distance: {distance_in_km} km")
        print(f"Duration: {duration_in_minutes} minutes")
    else:
        print(f"Error: API response status is {data['status']}.")
        if 'error_message' in data:
            print(f"Error Message: {data['error_message']}")
except requests.exceptions.RequestException as e:
    print(f"HTTP Request failed: {e}")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON response: {e}")


# Google Map API: actual drive time

In [ ]:
start_datetime = datetime.now()
print(f"Script started at: {start_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
start_time = time.time()
wait_time = random.uniform(2, 5)

In [ ]:
# Zone Shape Dataframe

zone_shape_df = client.query(f'''

SELECT
  zones.id,
  zones.name,
  zones.shape
FROM `fulfillment-dwh-production.curated_data_shared.countries` AS countries
LEFT JOIN countries.cities AS cities
LEFT JOIN cities.zones AS zones
WHERE countries.country_code = "tw"
  AND zones.is_active
  AND zones.name = "Bali"

''').to_dataframe()

zone_shape_df

In [ ]:
from shapely.wkt import loads
import pandas as pd

# Adjust Dataframe from string to polygon
zone_shape_df['shape'] = zone_shape_df['shape'].apply(loads)

In [ ]:
# Points Dataframe

point_df = client.query(f'''

SELECT
  *
FROM `foodpanda-tw-bigquery.test.zone_locations`
WHERE lg_zone_name = "Bali"

''').to_dataframe()

point_df

In [ ]:
# fp Active Vendor Dataframe

active_vendor_df = client.query(f'''

SELECT
  vendor_code,
  vendor_name,
  vendors.longitude,
  vendors.latitude
FROM `fulfillment-dwh-production.pandata_report.country_TW_vdr_svc_vendors` AS vendors
WHERE NOT vendors.is_testing
  AND vendors.is_restaurants
  AND vendors.is_active

''').to_dataframe()

active_vendor_df

In [ ]:
end_time = time.time()
execution_time_seconds = end_time - start_time
execution_time_minutes = execution_time_seconds / 60
print(f"Script completed in {execution_time_minutes:.2f} minutes.")

# Upload dataframe to GCP

In [ ]:
!pip install pandas_gbq==0.20.0

In [ ]:
import pandas as pd
from datetime import datetime
import re
import google.oauth2.credentials
import pandas_gbq
import json

In [ ]:
f = open('user_account.json')
credential = json.load(f)

credentials = google.oauth2.credentials.Credentials(
    credential['token'],
    refresh_token=credential['refresh_token'],
    token_uri=credential['token_uri'],
    client_id=credential['client_id'],
    client_secret=credential['client_secret']
    )

pandas_gbq.context.credentials = credentials

In [ ]:
project_id = "database_name"
table_id = 'table_name'

results_df.to_gbq(destination_table = table_id,
              project_id = project_id,
              credentials = credentials,
              if_exists='append')